In [2]:
import pandas as pd



In [3]:
'''
quota = pd.read_html('quota.html', match= "Quota")
print(len(quota))
df = quota[0]
df.head()
df.columns = ['Department', 'Status', "Quota", "Current","a"]
df.head()
df.drop("a",axis=1, inplace=True)
df
df.drop(labels=range(0,2), inplace=True)
df = df.reset_index(drop=True)
df
'''

'''
import requests
from bs4 import BeautifulSoup
 
 
url = 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
for link in soup.find_all('a'):
    print(link.get('href'))
'''

"\nimport requests\nfrom bs4 import BeautifulSoup\n \n \nurl = 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS'\nreqs = requests.get(url)\nsoup = BeautifulSoup(reqs.text, 'html.parser')\n \nurls = []\nfor link in soup.find_all('a'):\n    print(link.get('href'))\n"

In [4]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

driver1 = webdriver.Chrome()
driver1.get("https://registration.boun.edu.tr/buis/General/schedule.aspx?p=semester")

wait = WebDriverWait(driver1, 10)
element1 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="submit"]')))
element_id1 = element1.get_attribute('id')
go_button = wait.until(EC.presence_of_element_located((By.ID, element_id1)))


go_button.click()

driver1.implicitly_wait(3)

link_elements = driver1.find_elements(By.TAG_NAME, 'a')

urls = []

for link in link_elements:
    urls.append(link.get_attribute('href'))

driver1.quit()


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

dfs = []

for url in urls:
    driver.get(url)
    info_links = driver.find_elements(By.LINK_TEXT,"Info")
    for info_link in info_links:
        info_link.click()
        # Switch to the new window
        new_window = driver.window_handles[-1]
        driver.switch_to.window(new_window)
        try:
            # Find the table element
            table = driver.find_element(By.TAG_NAME, "table")
            # Convert the table content to a DataFrame
            df2 = pd.read_html(table.get_attribute('outerHTML'))[0]
            strongs = driver.find_elements(By.TAG_NAME, "strong")
            strongs_text = [strong.text for strong in strongs]
            strongs_text_repeated = np.repeat(strongs_text, len(df2))
            df2["Code"] = strongs_text_repeated[:len(df2)]
            dfs.append(df2)
        except IndexError:
            # No table found, move on to the next link
            pass
        finally:
            # Close the new window
            driver.close()
            # Switch back to the main window
            main_window = driver.window_handles[0]
            driver.switch_to.window(main_window)


result = pd.concat(dfs, ignore_index=True)

In [6]:
result = result[["Code",0,1,2,3,4]]

In [7]:
result.drop(result.columns[[5]], axis=1, inplace=True)

In [8]:
result.columns = ['Code', 'Department', 'Status', 'Quota', 'Current Quota']

In [9]:
words = ["Departmental Quotas:", "Department", "OZELLISANS", "PHD","MASTER","OZELMASTER","OZELPHD"]

result = result[result.Department.isin(words) == False]
result = result[result.Status.isin(words) == False]
result.reset_index(drop=True, inplace=True)
result

,Code,Department,Status,Quota,Current Quota
0,DSAI301.01,ALL,ALL,Consent Of Instructor,0
1,ASIA502.02,ALL,ALL,Consent Of Instructor,2
2,ASIA519.02,ALL,ALL,Consent Of Instructor,5
3,ASIA521.02,ALL,ALL,Consent Of Instructor,1
4,ASIA527.02,ALL,ALL,Consent Of Instructor,2
...,...,...,...,...,...
3062,FA 49F.01,ALL,ALL,Consent Of Instructor,39
3063,FA 49S.01,ALL,ALL,Consent Of Instructor,16
3064,LIT 311.01,ALL,ALL,Consent Of Instructor,0
3065,LIT 311.01,ALL,UNDERGRADUATE,40,65


In [16]:
testdata = pd.read_csv("data.csv")

'\ndef get_availability(row):\n    quota = row["Quota"]\n    current_quota = row["Current Quota"]\n    if int(current_quota) == int(quota):\n        return "Full"\n    elif isinstance(quota, str):\n        return "Available"\n    elif int(current_quota) > int(quota):\n        return "Try to get consent"\n    else:\n        return "Unknown"\n    \ntestdata["Availability"] = testdata.apply(get_availability, axis=1)\n'

In [39]:

def get_availability(row):
    quota = row['Quota']
    try:
        quota = int(quota)
    except ValueError:
        pass
    current_quota = int(row['Current Quota'])

        
    if isinstance(quota, int) and current_quota == quota:
        return "Full"
    elif isinstance(quota, int) and current_quota < quota:
        return "Available"
    elif isinstance(quota, int) and current_quota > quota:
        return "Try to get consent"
    elif isinstance(quota, str):
        return "Available"
    

testdata["Availability"] = testdata.apply(get_availability, axis=1)


In [40]:
testdata

,Code,Department,Status,Quota,Current Quota,Availability
0,DSAI301.01,ALL,ALL,Consent Of Instructor,0,Available
1,ASIA502.02,ALL,ALL,Consent Of Instructor,2,Available
2,ASIA519.02,ALL,ALL,Consent Of Instructor,5,Available
3,ASIA521.02,ALL,ALL,Consent Of Instructor,1,Available
4,ASIA527.02,ALL,ALL,Consent Of Instructor,2,Available
...,...,...,...,...,...,...
3062,FA 49F.01,ALL,ALL,Consent Of Instructor,39,Available
3063,FA 49S.01,ALL,ALL,Consent Of Instructor,16,Available
3064,LIT 311.01,ALL,ALL,Consent Of Instructor,0,Available
3065,LIT 311.01,ALL,UNDERGRADUATE,40,65,Try to get consent


In [42]:
'''
testdata.to_excel("testing6.xlsx", index=False)
'''